## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
pd.set_option('display.max_columns', None)

Lab | Random Forests

Apply the Random Forests algorithm but this time only by upscaling the data.
Use Feature Selections that you have learned in class to decide if you want to use all of the features (PCA, etc)
Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?

Lab | Final regression model in "Health Care for All" Case

At this point, we have created a model to predict who will make a donation and who won't. But, what about the ammount of money that each person will give? In this lab, subset those that made a donation and use that subset to create a model to predict how much money will they give.

Evaluate the result of your model and estimate how much better the result are for the bussiness in comparison with the naive scenario we discuss on Monday.

## Get Data

In [2]:
categorical = pd.read_csv('categorical.csv')

In [3]:
numerical = pd.read_csv('numerical.csv')

In [4]:
target = pd.read_csv('target.csv')

In [5]:
data = pd.concat([numerical, categorical, target], axis = 1)

In [6]:
data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3

In [7]:
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [8]:
data.shape

(95412, 339)

In [9]:
data.isna().sum().sum()

0

In [10]:
data.columns

Index(['TCODE', 'AGE', 'INCOME', 'WEALTH1', 'HIT', 'MALEMILI', 'MALEVET',
       'VIETVETS', 'WWIIVETS', 'LOCALGOV',
       ...
       'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM',
       'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM',
       'TARGET_B', 'TARGET_D'],
      dtype='object', length=339)

In [11]:
data_num = pd.concat([numerical, target], axis = 1)

In [12]:
data_num.shape

(95412, 317)

In [13]:
data_cat = pd.concat([categorical, target], axis = 1)

In [14]:
data_cat.shape

(95412, 24)

## Feature Selection

In [15]:
donations_data_cat = data_cat[data_cat['TARGET_B']==1] # data of people who have made donation
donations_data_cat

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
20,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,1,4.0
30,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,1,7.0
45,other,24,H,F,3,L,D,C,C,1,86,1,31,10,93,12,94,4,96,2,87,4,1,5.0
78,CA,13,H,F,2,L,F,A,S,1,90,1,28,7,90,1,95,3,95,11,90,1,1,13.0
93,GA,18,H,M,3,L,E,A,S,2,92,1,24,10,92,9,95,9,95,9,92,9,1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95298,CA,36,H,F,3,L,F,A,T,2,86,1,53,4,89,6,96,1,96,1,86,8,1,20.0
95309,CA,12,H,F,3,L,F,B,S,1,94,1,47,1,93,10,94,2,95,12,93,10,1,15.0
95398,WI,11,H,F,3,L,G,B,S,1,86,1,11,10,89,6,95,11,96,2,87,11,1,3.0
95403,other,49,H,F,2,L,F,D,R,2,90,1,40,1,90,3,93,12,96,1,90,3,1,10.0


In [16]:
donations_data_cat = donations_data_cat.drop(columns=['TARGET_B','TARGET_D'])
donations_data_cat

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
20,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1
30,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4
45,other,24,H,F,3,L,D,C,C,1,86,1,31,10,93,12,94,4,96,2,87,4
78,CA,13,H,F,2,L,F,A,S,1,90,1,28,7,90,1,95,3,95,11,90,1
93,GA,18,H,M,3,L,E,A,S,2,92,1,24,10,92,9,95,9,95,9,92,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95298,CA,36,H,F,3,L,F,A,T,2,86,1,53,4,89,6,96,1,96,1,86,8
95309,CA,12,H,F,3,L,F,B,S,1,94,1,47,1,93,10,94,2,95,12,93,10
95398,WI,11,H,F,3,L,G,B,S,1,86,1,11,10,89,6,95,11,96,2,87,11
95403,other,49,H,F,2,L,F,D,R,2,90,1,40,1,90,3,93,12,96,1,90,3


In [17]:
donations_data = data_num[data_num['TARGET_B']==1] # data of people who have made donation
donations_data

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B,TARGET_D
20,2,62.000000,3,8,10,2,25,40,27,11,4,1,9,2707,672,929,99,0,0,45,55,97,1,0,2,1,0,0,0,0,1,0,0,1,0,0,0,35,42,43,37,47,50,27,34,43,23,9,20,27,16,8,6,15,15,13,37,21,14,16,8,15,23,30,47,29,11,2,55,10,13,22,190,267,89,49,1,10,10,8,8,8,0,902,960,8,8,88,12,97,3,11,42,72,60,34,79,21,8,1,7,12,21,5,21,58,17,0,1,0,0,4,35,74,99,0,1,64,63,62,14,4,2,8,0,5,1,7,0,95,96,96,96,5120.0,107.0,613.0,468,501,540,580,20328,6,8,19,23,23,11,7,1,2,4,6,13,27,25,14,8,1,1,14,2,66,1,61,39,20,74,17,2,2,0,0,0,2,4,1,75,20,21,1,67,71,82,62,80,60,80,65,99,0,1,21,20,8,11,17,1,1,10,0,5,3,2,1,0,0,3,23,5,2,4,12,7,3,2,1,14,12,7,5,11,4,1,4,70,10,0,140,5,3,24,21,9,32,7,3,23,3,16,7,1,2,0,12,25,3,40,20,27,7,0,0,2,12,0,0,3,1,4,1,0,0,0,4,0,1,58,95,1,0,4,95,69,23,4,13,2,17,35,91,99,1,12,3,93,0,5,1,0,1,99,1,97,99,99,9,2,6,3,26,65,5,12,61.00,15,10,2.00,7.0,5.0,12,4.066667,82943,1,3,3,1,4.0
30,0,61.611649,5,9,0,1,37,58,16,8,1,5,9,2147,591,640,99,0,0,49,51,94,2,0,3,5,1,0,1,1,0,0,0,3,0,0,2,29,36,37,27,38,40,37,46,37,17,8,34,34,15,6,2,1,18,21,31,19,11,5,0,3,7,23,71,47,15,6,77,5,1,17,286,334,99,99,0,0,0,0,0,0,0,1002,1166,7,5,97,3,96,4,3,63,92,88,59,97,3,4,1,3,5,4,1,30,60,4,1,2,0,4,15,50,94,99,2,0,91,70,72,20,4,0,0,0,3,0,0,0,78,83,99,99,2920.0,201.0,618.0,633,638,652,663,19703,2,5,1,17,48,19,7,0,1,3,4,0,16,51,19,7,0,1,5,0,65,3,74,26,25,89,9,0,0,0,0,0,1,2,1,82,34,42,22,70,82,98,67,97,66,68,61,99,99,0,34,23,4,10,12,0,2,9,0,4,1,1,0,0,4,2,25,5,2,4,8,8,8,0,3,10,8,9,4,8,1,5,5,76,4,1,160,0,1,12,25,10,39,13,5,33,6,25,7,1,1,0,19,37,2,58,11,16,10,0,4,1,15,0,0,3,2,0,2,0,0,0,1,0,3,48,97,1,1,1,99,91,24,3,6,6,57,85,95,95,5,0,0,75,0,20,0,0,5,96,4,90,99,99,12,4,1,6,10,24,0,2,68.00,11,6,2.00,10.0,7.0,9,6.181818,190313,1,3,14,1,7.0
45,0,66.000000,5,9,5,0,33,24,39,6,5,1,9,2160,683,900,89,0,11,48,52,99,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,42,47,49,40,48,50,20,33,42,25,7,14,23,21,17,13,5,14,13,33,22,18,24,7,22,22,43,35,19,6,1,72,7,5,16,165,240,69,58,1,28,27,4,0,0,0,1282,1399,5,4,96,4,97,3,8,27,76,71,25,89,11,2,0,2,9,18,2,16,62,21,0,0,0,15,33,74,88,98,2,0,70,67,67,13,3,28,0,0,2,2,0,0,39,57,71,89,1360.0,173.0,637.0,550,637,607,703,26007,5,11,19,10,24,17,5,3,5,1,5,16,12,28,22,6,5,6,27,1,76,1,61,39,12,89,7,0,0,0,0,0,0,3,1,5,13,16,3,39,67,79,56,78,52,67,67,0,0,0,25,24,4,13,12,0,1,6,0,9,1,3,1,1,0,3,2

In [18]:
donations_data.shape

(4843, 317)

In [19]:
X = donations_data.drop(columns=['TARGET_B','TARGET_D']) # we split that into X (features of people who made donation) and y (target).


In [20]:
y = donations_data['TARGET_D']
y.shape

(4843,)

In [21]:
X.dtypes

TCODE         int64
AGE         float64
INCOME        int64
WEALTH1       int64
HIT           int64
             ...   
AVGGIFT     float64
CONTROLN      int64
HPHONE_D      int64
RFA_2F        int64
CLUSTER2      int64
Length: 315, dtype: object

In [22]:
y.dtypes

dtype('float64')

In [23]:
X_added_constant = sm.add_constant(X)
X_added_constant
model = sm.OLS(y,X_added_constant).fit() # linear regression model but sm.OLS: ordinary least squares. same mathematical method
model.summary() # OLS does require an additonal feature column. constant column that has a value of 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               TARGET_D   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     16.56
Date:                Thu, 20 Oct 2022   Prob (F-statistic):               0.00
Time:                        22:23:41   Log-Likelihood:                -17226.
No. Observations:                4843   AIC:                         3.508e+04
Df Residuals:                    4527   BIC:                         3.713e+04
Df Model:                         315                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         15.9657      2.375      6.723      0.000      11.310      20.621
TCODE      -4.046e-05      0.000     -0.359      0.720      -0.000       0.000
AGE            0.0169      0.010      1.655      0.098      -0.003       0.037
INCOME         0.0541      0.090      0.599      0.549      -0.123       0.231
WEALTH1        0.0521      0.063      0.823      0.411      -0.072       0.176
HIT           -0.0220      0.014     -1.522      0.128      -0.050       0.006
MALEMILI      -0.0786      0.050     -1.564      0.118      -0.177       0.020
MALEVET        0.0005      0.026      0.020      0.984      -0.051       0.052
VIETVETS      -0.0100      0.017     -0.594      0.552      -0.043       0.023
WWIIVETS      -0.0363      0.017     -2.143      0.032      -0.069      -0.003
LOCALGOV       0.0945      0.060      1.572      0.116      -0.023       0.212
STATEGOV       0.0306      0.059      0.517      0.605      -0.085       0.146
FEDGOV         0.1504      0.082      1.839      0.066      -0.010       0.311
WEALTH2       -0.0008      0.061     -0.013      0.990      -0.120       0.118
POP901        -0.0003      0.000     -1.340      0.180      -0.001       0.000
POP902         0.0003      0.001      0.351      0.725      -0.001       0.002
POP903         0.0005      0.000      1.138      0.255      -0.000       0.001
POP90C1        0.6152      0.350      1.758      0.079      -0.071       1.301
POP90C2        0.6131      0.349      1.759      0.079      -0.070       1.296
POP90C3        0.6166      0.350      1.763      0.078      -0.069       1.302
POP90C4       -1.5998      4.544     -0.352      0.725     -10.509       7.310
POP90C5       -1.7343      4.543     -0.382      0.703     -10.642       7.173
ETH1           0.7087      0.228      3.105      0.002       0.261       1.156
ETH2          -0.2149      0.282     -0.763      0.445      -0.767       0.337
ETH3           0.0820      0.073      1.125      0.261      -0.061       0.225
ETH4           0.1285      0.147      0.875      0.382      -0.159       0.416
ETH5          -0.6731      0.246     -2.732      0.006      -1.156      -0.190
ETH6          -0.2976      0.282     -1.054      0.292      -0.851       0.256
ETH7          -0.0102      0.168     -0.061      0.951      -0.340       0.319
ETH8          -0.0632      0.150     -0.421      0.673      -0.357       0.231
ETH9          -0.1884      0.152     -1.241      0.215      -0.486       0.109
ETH10         -0.0545      0.180     -0.303      0.762      -0.408       0.299
ETH11         -0.2751      0.220     -1.252      0.211      -0.706       0.156
ETH12         -0.0136      0.201     -0.068      0.946      -0.407       0.380
ETH13          0.6936      0.246      2.819      0.005       0.211       1.176
ETH14          0.7806      0.272      2.874      0.004       0.248       1.313
ET

In [24]:
cols_keep = model.pvalues[model.pvalues <= 0.05]

In [25]:
cols_drop = []

In [26]:
for i in X:
    if model.pvalues[i] > 0.05:
        cols_drop.append(i)
        

In [27]:
cols_drop

['TCODE',
 'AGE',
 'INCOME',
 'WEALTH1',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'WEALTH2',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN3',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW1',
 'DW2',
 'DW3',
 'DW4',
 'DW5',
 'DW6',
 'DW7',
 'DW8',
 'DW9',
 'HV1',
 'HV2',
 'HV3',
 'HV4',
 'HU1',
 'HU2',
 'HU3',
 'HU4',
 'HU5',
 'HHD1',
 'HHD2',
 'HHD3',
 'HHD4',
 'HHD5',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HVP1',
 'HVP2'

In [28]:
X = X.drop(columns=cols_drop)

In [29]:
X.isna().sum().sum()

0

In [30]:
X.columns

Index(['WWIIVETS', 'ETH1', 'ETH5', 'ETH13', 'ETH14', 'ETH15', 'ETH16', 'ETHC1',
       'ETHC2', 'ETHC3', 'HUR1', 'HUPA4', 'HUPA5', 'HUPA6', 'HUPA7', 'RP3',
       'DMA', 'HC17', 'HC19', 'HC20', 'AC2', 'CARDPM12', 'RAMNTALL',
       'NGIFTALL', 'MINRAMNT', 'LASTGIFT', 'TIMELAG', 'AVGGIFT', 'CONTROLN',
       'RFA_2F'],
      dtype='object')

In [31]:
X.shape

(4843, 30)

In [32]:
categorical.shape

(95412, 22)

In [33]:
X = pd.concat([X, donations_data_cat], axis = 1)

In [34]:
X.shape

(4843, 52)

In [35]:
# data['GENDER'].value_counts()

In [36]:
# y = data['TARGET_B']
# y.shape

In [37]:
# target_d = data['TARGET_D']

In [38]:
# X = data.drop(['TARGET_B','TARGET_D'], axis=1)

## Train test split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [40]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (3874, 52)
X_test shape:  (969, 52)
y_train shape:  (3874,)
y_test shape:  (969,)


In [41]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [42]:
print('X_train_num shape: ', X_train_num.shape)
print('X_test_num shape: ', X_test_num.shape)
print('X_train_cat shape: ', X_train_cat.shape)
print('X_test_cat shape: ', X_test_cat.shape)

X_train_num shape:  (3874, 45)
X_test_num shape:  (969, 45)
X_train_cat shape:  (3874, 7)
X_test_cat shape:  (969, 7)


In [43]:
# MinMax scaler to normalize the features
transformer = MinMaxScaler().fit(X_train_num)

In [44]:
X_train_norm1 = transformer.transform(X_train_num)
X_train_scale = pd.DataFrame(X_train_norm1,columns=X_train_num.columns)
X_train_scale.head()

,WWIIVETS,ETH1,ETH5,ETH13,ETH14,ETH15,ETH16,ETHC1,ETHC2,ETHC3,HUR1,HUPA4,HUPA5,HUPA6,HUPA7,RP3,DMA,HC17,HC19,HC20,AC2,CARDPM12,RAMNTALL,NGIFTALL,MINRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,RFA_2F,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,0.474747,0.989899,0.020408,0.000000,0.035714,0.0,0.017544,0.682927,0.648352,0.113402,0.000000,0.084211,0.028169,0.000000,0.000000,0.898990,0.568672,0.010101,0.010101,1.000000,0.161290,0.117647,0.020634,0.133333,0.006667,0.029133,0.045455,0.003731,0.979851,0.000000,0.673077,1.0,0.333333,0.454545,0.0,0.000000,0.090909,0.727273,0.909091,0.818182,0.909091,0.0,0.909091,0.722222,0.181818
1,0.080808,0.979798,0.010204,0.010870,0.000000,0.0,0.000000,0.682927,0.703297,0.051546,0.000000,0.052632,0.028169,0.000000,0.000000,0.979798,0.740068,1.000000,1.000000,1.000000,0.161290,0.352941,0.052571,0.066667,0.022222,0.042222,0.068182,0.029090,0.676623,0.333333,0.326923,1.0,0.333333,0.545455,0.0,0.571429,0.000000,0.545455,0.818182,0.818182,0.727273,0.0,0.727273,0.777778,0.818182
2,0.282828,0.656566,0.214286,0.152174,0.035714,0.0,0.105263,0.365854,0.461538,0.082474,0.088889,0.073684,0.070423,0.272727,0.021739,0.595960,0.896708,0.979798,0.929293,0.979798,0.161290,0.294118,0.005714,0.000000,0.055556,0.055556,0.204545,0.050787,0.728942,0.000000,0.634615,1.0,0.000000,0.909091,0.0,0.582418,0.000000,0.909091,0.090909,0.909091,0.090909,0.5,0.090909,1.000000,0.090909
3,0.414141,0.838384,0.173469,0.173913,0.000000,0.0,0.017544,0.439024,0.450549,0.247423,0.044444,0.147368,0.225352,0.070707,0.000000,0.313131,0.919410,1.000000,1.000000,1.000000,0.193548,0.352941,0.054286,0.111111,0.011111,0.040000,0.295455,0.017285,0.960838,0.333333,0.500000,1.0,0.333333,0.000000,0.0,0.153846,0.454545,0.272727,0.545455,0.909091,0.000000,0.5,0.000000,0.444444,0.727273
4,0.282828,0.969697,0.020408,0.010870,0.000000,0.0,0.000000,0.536585,0.714286,0.092784,0.011111,0.084211,0.000000,0.000000,0.000000,0.838384,0.929625,0.909091,0.050505,0.989899,0.225806,0.352941,0.017714,0.044444,0.011111,0.035556,0.204545,0.015498,0.934796,0.000000,0.750000,1.0,0.333333,0.636364,0.0,0.494505,0.000000,0.636364,0.909091,0.818182,0.818182,0.0,0.818182,0.833333,0.090909


In [45]:
X_train_scale.shape

(3874, 45)

In [46]:
X_test_norm1 = transformer.transform(X_test_num)
X_test_scale = pd.DataFrame(X_test_norm1,columns=X_test_num.columns)
X_test_scale.head()

,WWIIVETS,ETH1,ETH5,ETH13,ETH14,ETH15,ETH16,ETHC1,ETHC2,ETHC3,HUR1,HUPA4,HUPA5,HUPA6,HUPA7,RP3,DMA,HC17,HC19,HC20,AC2,CARDPM12,RAMNTALL,NGIFTALL,MINRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,RFA_2F,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,0.333333,0.383838,0.030612,0.010870,0.035714,0.0,0.035088,0.048780,0.296703,0.092784,0.377778,0.031579,0.084507,0.474747,0.000000,0.939394,0.844495,1.000000,1.000000,1.0,0.129032,0.235294,0.002857,0.000000,0.044444,0.044444,0.204545,0.039619,0.908238,0.000000,0.000000,0.5,0.000000,0.909091,0.0,0.351648,0.000000,0.909091,0.000000,0.909091,0.000000,0.5,0.000000,1.000000,0.000000
1,0.525253,0.979798,0.051020,0.043478,0.000000,0.0,0.017544,0.414634,0.516484,0.340206,0.200000,0.042105,0.028169,0.040404,0.217391,0.484848,0.854711,0.888889,0.131313,1.0,0.290323,0.294118,0.073143,0.211111,0.006667,0.033333,0.113636,0.010920,0.709809,0.000000,0.865385,0.5,0.333333,0.000000,0.0,0.000000,0.090909,0.090909,0.454545,0.818182,0.272727,0.0,0.272727,0.444444,0.909091
2,0.535354,1.000000,0.010204,0.010870,0.000000,0.0,0.000000,0.536585,0.582418,0.247423,0.011111,0.178947,0.000000,0.000000,0.065217,0.040404,0.684449,0.979798,0.797980,1.0,0.193548,0.235294,0.005714,0.011111,0.011111,0.044444,0.272727,0.022869,0.549203,0.000000,0.846154,1.0,0.333333,0.818182,0.0,0.527473,0.000000,0.727273,1.000000,0.818182,1.000000,0.0,1.000000,0.888889,1.000000
3,0.757576,1.000000,0.010204,0.000000,0.000000,0.0,0.000000,0.121951,0.296703,0.690722,0.022222,0.052632,0.014085,0.020202,0.021739,0.848485,0.611805,0.959596,0.949495,1.0,0.548387,0.294118,0.037143,0.188889,0.004444,0.011111,0.590909,0.004877,0.231312,0.666667,0.538462,1.0,0.333333,0.000000,0.0,0.164835,0.090909,0.181818,1.000000,0.545455,0.909091,0.5,0.181818,0.444444,0.818182
4,0.323232,0.989899,0.000000,0.000000,0.000000,0.0,0.000000,0.487805,0.637363,0.195876,0.044444,0.115789,0.056338,0.040404,0.065217,0.080808,0.736663,0.747475,0.545455,1.0,0.225806,0.352941,0.006857,0.055556,0.006667,0.008889,0.113636,0.005001,0.333525,1.000000,0.846154,1.0,0.333333,0.818182,0.0,0.461538,0.000000,0.818182,0.909091,0.818182,0.818182,0.5,0.181818,0.944444,0.000000


In [47]:
X_test_scale.shape

(969, 45)

In [48]:
# Encode the categorical features using One-Hot Encoding

In [49]:
X_train_cat.dtypes

STATE       object
HOMEOWNR    object
GENDER      object
RFA_2R      object
RFA_2A      object
GEOCODE2    object
DOMAIN_A    object
dtype: object

In [50]:
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3874 entries, 88982 to 53887
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   STATE     3874 non-null   object
 1   HOMEOWNR  3874 non-null   object
 2   GENDER    3874 non-null   object
 3   RFA_2R    3874 non-null   object
 4   RFA_2A    3874 non-null   object
 5   GEOCODE2  3874 non-null   object
 6   DOMAIN_A  3874 non-null   object
dtypes: object(7)
memory usage: 242.1+ KB


In [51]:
X_train_cat = X_train_cat.astype('str')
X_train_cat

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
88982,other,H,F,L,E,A,T
24612,other,H,M,L,F,B,S
62247,other,H,F,L,G,D,T
17784,WA,H,F,L,F,C,C
18435,WA,H,F,L,F,A,T
...,...,...,...,...,...,...,...
20647,NC,H,F,L,G,B,C
64107,other,U,F,L,F,C,C
32452,other,H,M,L,F,B,T
51732,other,H,F,L,F,B,S


In [52]:
encoder = OneHotEncoder(drop='first',handle_unknown = 'ignore').fit(X_train_cat)

cols = encoder.get_feature_names(input_features=X_train_cat.columns)

X_train_cat_encode = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)

X_train_cat_encode.head(10)

/Users/ozguryildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [53]:
X_train_cat_encode.shape

(3874, 24)

In [54]:
X_test_cat = X_test_cat.astype('str')

In [55]:
X_test_cat

,STATE,HOMEOWNR,GENDER,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
31771,other,U,F,L,F,B,U
80425,other,U,F,L,F,C,R
61896,other,H,M,L,F,D,R
36289,FL,H,F,L,D,B,C
30894,IN,H,M,L,D,D,R
...,...,...,...,...,...,...,...
11536,IN,H,F,L,F,D,T
48736,other,H,F,L,F,C,T
22224,MI,H,M,L,G,D,R
244,CA,H,M,L,F,A,S


In [56]:
cols = encoder.get_feature_names(input_features=X_test_cat.columns)

X_test_cat_encode = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)

X_test_cat_encode.head(10)

/Users/ozguryildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [57]:
X_test_cat_encode.shape

(969, 24)

In [58]:
X_train = pd.concat([X_train_cat_encode, X_train_scale], axis=1)

X_test = pd.concat([X_test_cat_encode, X_test_scale], axis=1)

In [59]:
X_train.index
y_train.reset_index(drop = True, inplace = True)

In [60]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (3874, 69)
X_test shape:  (969, 69)
y_train shape:  (3874,)
y_test shape:  (969,)


In [61]:
data = pd.concat([X_train, y_train], axis = 1)
data

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,WWIIVETS,ETH1,ETH5,ETH13,ETH14,ETH15,ETH16,ETHC1,ETHC2,ETHC3,HUR1,HUPA4,HUPA5,HUPA6,HUPA7,RP3,DMA,HC17,HC19,HC20,AC2,CARDPM12,RAMNTALL,NGIFTALL,MINRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,RFA_2F,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_D
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.474747,0.989899,0.020408,0.000000,0.035714,0.000000,0.017544,0.682927,0.648352,0.113402,0.000000,0.084211,0.028169,0.000000,0.000000,0.898990,0.568672,0.010101,0.010101,1.000000,0.161290,0.117647,0.020634,0.133333,0.006667,0.029133,0.045455,0.003731,0.979851,0.000000,0.673077,1.0,0.333333,0.454545,0.0,0.000000,0.090909,0.727273,0.909091,0.818182,0.909091,0.0,0.909091,0.722222,0.181818,20.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.080808,0.979798,0.010204,0.010870,0.000000,0.000000,0.000000,0.682927,0.703297,0.051546,0.000000,0.052632,0.028169,0.000000,0.000000,0.979798,0.740068,1.000000,1.000000,1.000000,0.161290,0.352941,0.052571,0.066667,0.022222,0.042222,0.068182,0.029090,0.676623,0.333333,0.326923,1.0,0.333333,0.545455,0.0,0.571429,0.000000,0.545455,0.818182,0.818182,0.727273,0.0,0.727273,0.777778,0.818182,20.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.282828,0.656566,0.214286,0.152174,0.035714,0.000000,0.105263,0.365854,0.461538,0.082474,0.088889,0.073684,0.070423,0.272727,0.021739,0.595960,0.896708,0.979798,0.929293,0.979798,0.161290,0.294118,0.005714,0.000000,0.055556,0.055556,0.204545,0.050787,0.728942,0.000000,0.634615,1.0,0.000000,0.909091,0.0,0.582418,0.000000,0.909091,0.090909,0.909091,0.090909,0.5,0.090909,1.000000,0.090909,25.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.414141,0.838384,0.173469,0.173913,0.000000,0.000000,0.017544,0.439024,0.450549,0.247423,0.044444,0.147368,0.225352,0.070707,0.000000,0.313131,0.919410,1.000000,1.000000,1.000000,0.193548,0.352941,0.054286,0.111111,0.011111,0.040000,0.295455,0.017285,0.960838,0.333333,0.500000,1.0,0.333333,0.000000,0.0,0.153846,0.454545,0.272727,0.545455,0.909091,0.000000,0.5,0.000000,0.444444,0.727273,10.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.282828,0.969697,0.020408,0.010870,0.000000,0.000000,0.000000,0.536585,0.714286,0.092784,0.011111,0.084211,0.000000,0.000000,0.000000,0.838384,0.929625,0.909091,0.050505,0.989899,0.225806,0.352941,0.017714,0.044444,0.011111,0.035556,0.204545,0.015498,0.934796,0.000000,0.750000,1.0,0.333333,0.636364,0.0,0.494505,0.000000,0.636364,0.909091,0.818182,0.818182,0.0,0.818182,0.833333,0.090909,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242424,0.666667,0.010204,0.000000,0.000000,0.000000,0.000000,0.292683,0.483516,0.092784,0.011111,0.073684,0.042254,0.262626,0.000000,0.969697,0.635641,1.000000,1.000000,1.000000,0.129032,0.352941,0.048571,0.033333,0.055556,0.055556,0.068182,0.050787,0.092711,0.000000,0.442308,1.0,0.000000,0.727273,0.0,0.428571,0.000000,0.818182,1.000000,0.636364,0.818182,0.0,1.000000,0.833333,0.818182,25.0
3870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.585859,0.989899,0.010204,0.010870,0.000000,0.000000

In [62]:
data.shape

(3874, 70)

In [63]:
data.isna().sum().sum()

0

In [64]:
from sklearn import preprocessing
from sklearn import utils

print(utils.multiclass.type_of_target(y_train))

continuous


In [65]:
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [66]:
print(utils.multiclass.type_of_target(y_train))

multiclass


In [68]:
print(utils.multiclass.type_of_target(y_test))

continuous


In [69]:
y_test = label_encoder.fit_transform(y_test)

In [70]:
print(utils.multiclass.type_of_target(y_test))

multiclass


## RandomForestClassifier

In [72]:
clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             #max_samples=0.2, # fraction of X-train to use in each tree
                             )
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
# display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.38513164687661333
0.018575851393188854


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [73]:
# For cross validation
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.8)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

/Users/ozguryildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


0.3727390180878553


In [74]:
cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')

/Users/ozguryildirim/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


array([0.04773562, 0.04674863, 0.04433271, 0.04469722, 0.05116395])

## DecisionTreeRegressor

In [75]:
regr = DecisionTreeRegressor(max_depth=5)
model = regr.fit(X_train, y_train)

In [76]:
print("test data accuracy was: ",regr.score(X_test, y_test))
print("train data accuracy was: ",regr.score(X_train, y_train))

test data accuracy was:  -0.029960886184164925
train data accuracy was:  0.6579694279873558


In [77]:
regr = DecisionTreeRegressor(max_depth=10,
                             criterion = 'squared_error',
                             min_samples_split=2,
                             min_samples_leaf = 1,
                             max_features = 6)
model = regr.fit(X_train, y_train)
print("test data accuracy was: ",regr.score(X_test, y_test))
print("train data accuracy was: ",regr.score(X_train, y_train))

test data accuracy was:  -0.38923141740023937
train data accuracy was:  0.7255186834256205


In [78]:
regr = DecisionTreeRegressor(max_depth=10,
                             criterion = 'squared_error',
                             min_samples_split=10,
                             min_samples_leaf = 10,
                             max_features = 3)
model = regr.fit(X_train, y_train)
print("test data accuracy was: ",regr.score(X_test, y_test))
print("train data accuracy was: ",regr.score(X_train, y_train))

test data accuracy was:  -0.2723224009040879
train data accuracy was:  0.5204059228833444


### Cross Validation

In [79]:
regr = DecisionTreeRegressor(max_depth=10,
                             criterion = 'squared_error',
                             min_samples_split=2,
                             min_samples_leaf = 1,
                             max_features = 6)

In [80]:
results = cross_validate(regr,X_train, y_train, cv = 5)

In [81]:
X_train.shape

(3874, 69)

In [82]:
results

{'fit_time': array([0.01042199, 0.00799489, 0.00875401, 0.0083971 , 0.01148391]),
 'score_time': array([0.00235772, 0.00232697, 0.00224614, 0.00246096, 0.00220323]),
 'test_score': array([0.3692001 , 0.39839548, 0.30263673, 0.28944193, 0.40825741])}

In [83]:
print(results['test_score'])
print(results['test_score'].mean())

[0.3692001  0.39839548 0.30263673 0.28944193 0.40825741]
0.353586330527394


In [84]:
regr = DecisionTreeRegressor(max_depth=10,
                             criterion = 'squared_error',
                             min_samples_split=10,
                             min_samples_leaf = 10,
                             max_features = 3)

In [85]:
results = cross_validate(regr,X_train, y_train, cv = 5)
print(results['test_score'])
print(results['test_score'].mean())

[0.50520118 0.34122102 0.4330558  0.29989026 0.27072863]
0.37001937741012864


### Hyperparameter search

### Grid Search

In [86]:
# function GridSearchCV with cross validation

max_depth_choices= [3,10,None]
criterion_choices = ['squared_error','absolute_error']
min_samples_split_choices = [2,10]
min_samples_leaf_choices = [2,10]        

In [87]:
# Create the  grid 
# this is a dictionary from hyperparameters to potential values
# the keys in this dictionary have to match the names of the hyperparameters in the documentation of the model
grid = {'max_depth': max_depth_choices,
        'criterion': criterion_choices,
        'min_samples_split': min_samples_split_choices,
        'min_samples_leaf': min_samples_leaf_choices}

In [88]:
# Instantiate the grid search model object

# estimator -> model to optimize 
model = DecisionTreeRegressor()
# param_grid -> state the dictionary of parameters to optimize
# cv = 5 -> number of cross validation folds <------ CV is REALLY important in grid search. Why?
grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = 5)

In [89]:
%%time
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

CPU times: user 2min 17s, sys: 272 ms, total: 2min 17s
Wall time: 2min 18s


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [3, 10, None],
                         'min_samples_leaf': [2, 10],
                         'min_samples_split': [2, 10]})

In [90]:
# and the winner is...
grid_search.best_params_

{'criterion': 'squared_error',
 'max_depth': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 2}

In [91]:
# in grid search you are more likely to get really good results in your training set, even with CV
grid_search.best_score_

0.5897308542798735

### Random Search

In [92]:
max_depth_choices= [3,4,5,6,7,8,9,10,None]
criterion_choices = ['squared_error','absolute_error']
min_samples_split_choices = [2,3,4,5,6,7,8,9,10]
min_samples_leaf_choices = [2,3,4,5,6,7,8,9,10]
max_features_choices = [2,3,4,5,6]

random_grid = {'max_depth': max_depth_choices,
               'criterion': criterion_choices,
               'min_samples_split': min_samples_split_choices,
               'min_samples_leaf': min_samples_leaf_choices,
               'max_features': max_features_choices}

In [93]:
%%time
# trying grid search
# means building 9 * 2 * 9 * 9 * 5 * 5 = 36450 models

model = DecisionTreeRegressor()
grid_search = GridSearchCV(estimator = model, param_grid = random_grid, cv = 5, n_jobs=-1)
grid_search.fit(X_train, y_train)


CPU times: user 46 s, sys: 6.5 s, total: 52.5 s
Wall time: 20min 36s


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
                         'max_features': [2, 3, 4, 5, 6],
                         'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [94]:
grid_search.best_params_

{'criterion': 'squared_error',
 'max_depth': 6,
 'max_features': 6,
 'min_samples_leaf': 6,
 'min_samples_split': 5}

In [95]:
grid_search.best_score_

0.5216217603660616

In [96]:
#and now more realistic
model = DecisionTreeRegressor()
random_search = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter=100, cv = 5, n_jobs = 10)


In [97]:
%%time
random_search.fit(X_train,y_train)

CPU times: user 1.23 s, sys: 183 ms, total: 1.42 s
Wall time: 20.8 s


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=100,
                   n_jobs=10,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                                      None],
                                        'max_features': [2, 3, 4, 5, 6],
                                        'min_samples_leaf': [2, 3, 4, 5, 6, 7,
                                                             8, 9, 10],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10]})

In [98]:
random_search.best_params_

{'min_samples_split': 5,
 'min_samples_leaf': 9,
 'max_features': 6,
 'max_depth': None,
 'criterion': 'absolute_error'}

In [99]:
random_search.best_score_

0.4889705565517925